In [ ]:
"""Functions for plugging into Pipulate-frameworks for conducting SEO investigations."""

In [ ]:
import requests, re
from collections import namedtuple
Response = namedtuple('Response', 'ok status text')

In [ ]:
def Title(**row_dict):
    pattern = re.compile(r'<title\s?>(.*?)</title\s?>', re.DOTALL)
    matches = pattern.findall(row_dict['response'].text)
    if matches:
        return Response(True, '200', matches[0].strip())
    else:
        return Response(True, '200', None)

In [ ]:
if __name__ == '__main__':
    test_url = 'http://mikelev.in/'
    response = requests.get(test_url)
    print(Title(url=test_url, response=response))